In [2]:
import os
import sys
from dotenv import load_dotenv
from pathlib import Path

if sys.platform == 'linux':
    load_dotenv(dotenv_path=Path('.') / '.env.linux')
elif sys.platform == 'win32':
    load_dotenv(dotenv_path=Path('.') / '.env.win')
else:
    raise ValueError('Ваша операционная система не поддерживается')

os.environ['HF_HOME'] = os.getenv('HUGGING_FACE_CACHE_DIR')
DATASET_PATH = os.getenv('DATASET_PATH', None)
print(DATASET_PATH)
DATASET_FOLDER = os.getenv('DATASET_FOLDER_PATH', None)
MODEL_NAME = os.getenv('MODEL_NAME', None)
MODEL_SIZE = os.getenv('MODEL_SIZE', None)
MODEL_SAVE_DIR=os.getenv('MODEL_SAVE_DIR', None)
print(MODEL_NAME)

/mnt/12A4CA9DA4CA8329/Files/Datasets/recipes_generation/fine_tuning_preprocessed.csv
ai-forever/rugpt3small_based_on_gpt2


In [3]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, AutoTokenizer, DataCollatorForLanguageModeling, AutoModel
from transformers import Trainer, TrainingArguments

from datasets import load_from_disk, DatasetDict

pad_token = '<PAD>'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {DEVICE}')

DEVICE: cuda


In [3]:
model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# У GPT2 не было паддинг токена по умолчанию
tokenizer.pad_token = tokenizer.eos_token

# Количество параметров модели
model_size = sum(t.numel() for t in model.parameters())
print(f"model size: {model_size/1000**2:.1f}M parameters")

model size: 125.2M parameters


In [4]:
prepared_dataset = load_from_disk(Path(DATASET_FOLDER) / 'prepared_dataset_hg_format')
prepared_dataset = prepared_dataset.map(remove_columns=['name', 'instructions', 'ingredients', 'composition_list', 'prompt', 'length'])

In [5]:
before_main_train_dataset = DatasetDict({
    # Перемешиваем train‑датасет и берём первые 5000 строк
    'train': prepared_dataset['train'].shuffle(seed=42).take(5000),

    'test': prepared_dataset['test'].shuffle(seed=42).take(500)
})

In [6]:
# data_collator отвечает за создание батчей, их выравнивание при помощи паддинга, а также создает метки для входов 
# Сдвиг входов и меток для их выравнивания происходит внутри модели, поэтому коллатор данных просто копирует входы для создания меток
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False) # Отключаем masked language modelling (mlm)

out = data_collator([before_main_train_dataset["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 635])
attention_mask shape: torch.Size([5, 635])
labels shape: torch.Size([5, 635])


In [7]:
args = TrainingArguments(
    output_dir=os.path.join(MODEL_SAVE_DIR, 'training_checkpoints'),  # Кастомная директория
    per_device_train_batch_size=26,
    per_device_eval_batch_size=26,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    weight_decay=0.01,
    warmup_steps=300,
    lr_scheduler_type="cosine",
    learning_rate=1e-5,
    save_steps=200,
    eval_steps=200,
    logging_steps=50,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,  # Загружать лучшую модель
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=torch.cuda.is_available(),  # Только если GPU поддерживает
    gradient_checkpointing=True,  # Экономия памяти
    save_total_limit=3,  # Хранить только 3 лучших чекпоинта
    push_to_hub=False,
    report_to="none",  # Отключить логирование в сторонние сервисы
)

In [8]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=prepared_dataset["train"],
    eval_dataset=prepared_dataset["test"],
)

/tmp/ipykernel_13271/2468702332.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
# Обучение
print("Начало обучения...")
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Начало обучения...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
200,1.081200,0.827657
400,0.786700,0.749392
600,0.758600,0.737586


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=711, training_loss=2.3161195736226485, metrics={'train_runtime': 3420.961, 'train_samples_per_second': 21.571, 'train_steps_per_second': 0.208, 'total_flos': 2.3951389229568e+16, 'train_loss': 2.3161195736226485, 'epoch': 3.0})

In [ ]:
# Сохранение финальной модели
trainer.save_model(os.path.join(MODEL_SAVE_DIR, f'final_gpt2_{MODEL_SIZE}'))
tokenizer.save_pretrained(os.path.join(MODEL_SAVE_DIR, f'final_gpt2_{MODEL_SIZE}'))

print(f'Модель сохранена в {os.path.join(MODEL_SAVE_DIR, f'final_gpt2_{MODEL_SIZE}')}')

Модель сохранена в /mnt/2FADF63B267AA05B/AI_models/recepies_generation


In [4]:
import torch
from transformers import pipeline

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
pipe = pipeline("text-generation", model=os.path.join(MODEL_SAVE_DIR, f'final_gpt2_{MODEL_SIZE}'), device=device)

Device set to use cuda


In [16]:
txt = """Рецепт пельменей"""
print(pipe(txt, num_return_sequences=1)[0]["generated_text"])

Рецепт пельменей

Ингридиенты:
 Говяжий фарш 41.5 гр.
 Лук репчатый 0.2 гол.
 Оливковое масло 0.1 столов.
 Яйцо куриное 0.2 шт.
 Свежие дрожжи 0.1 чайн.
 Майонез 0.3 столов.
 Кумин 0.8 столов.
 Соль 0.2 чайн.
 Сушеный эстрагон 0.2 столов.
 Сметана 57.5 гр.
 Кумин (кориандр) 0.2 столов.
 Сливочное масло 16.5 гр.
 Пшеничная мука 0.1 столов.

Процесс приготовления:
1. Нарезать лук и обжарить его на растительном масле до состояния пюре.
2. Натереть на мелкой терке чеснок и смешать с майонезом, яйцом и сметаной.
3. Сливочное масло растопить в сковороде на среднем огне и добавить к луку и обжаривать его на среднем огне до золотистого цвета.
4. Смешать муку и растительное масло, добавить соль и перец, вымесить тесто.
5. Тесто разделить на 2 части и раскатать в пласт толщиной 0
